# 2.0 Preprocessing Datasets

[![Static Badge](https://img.shields.io/badge/Back_to_README.md-red?style=for-the-badge&logo=github&labelColor=black)](https://github.com/izzad2413/sustainable_ota)

### Table of Contents<a id="main"></a>

- [2.1 Duplicates & Error Data](#1)
- [2.2 Change NaN Value](#2)
- 2.3 Features Cleaning
  - [2.3.1 About](#3)
    - name & description
    - address
    - coordinate
  - [2.3.2 Room & Price](#4)
    - prices_range
  - [2.3.3 Review](#5)
    - review_titles & reviews_overall_score
    - score_status
    - count_qna
    - count_reviewer_by_cat & count_reviewer_by_scores
    - count_reviewer_by_lan
  - [2.3.4 Host](#6)
    - host_count_reviews
  - [2.3.5 Surroundings](#7)
    - landmarks, topattractions, publictransport, closestairports & restaurantscafes
  - [2.3.6 Facilities](#8)
  - [2.3.7 Sustainability](#9)
    - sustainable_level
    - sustainable impact category
- [2.4 Finalised Dataset](#10)
  - 2.4.1 Dropping the original features
  - 2.4.2 Move the target feature to the end
  - 2.4.3 Final check for null values
  - 2.4.4 Saving the processed data
- [2.5 Process Dataset for Modeling](#11)
  - 2.5.1 Encoding categorical variables
  - 2.5.2 Handling outliers
  - 2.5.3 Saving the processed data

In [32]:
import pandas as pd
import numpy as np
import re
from scipy.stats import mstats

In [2]:
data_dir = './../data/interim/combined_data_v4.csv' # dataset directory
df = pd.read_csv(data_dir) # read the dataset

### 2.1 Duplicates & Error Data<a id="1"></a> [![Static Badge](https://img.shields.io/badge/Back_to_Table_of_Contents-orange?style=flat&logo=jupyter&logoColor=white)](#main)

To scan any samples that are duplicated and may contain errors.

In [3]:
print("Before Drop")
print(f'Dataset size {len(df)}')
print(f'There are {df.duplicated().sum()} duplicated data detected\n')

# keep the first occurence, delete the others
# drop duplicates in a specific column, keeping the first occurrence
df.drop_duplicates(subset=['name'], keep='first', inplace=True)

# checking any error (null values) on sample
print("Data Checking")
condition_1 = df[(df['preferred_partner'] == 0) &
                 (df['state'].notnull()) &
                 (df['name'].isnull()) &
                 (df['address'].isnull())]

condition_2 = df[(df['name'] == 'testing genting ion delemen')]

condition_1_total_samples = len(condition_1)
condition_2_total_samples = len(condition_2)

# check the samples
print(f'Total samples that meet the condition 1: {condition_1_total_samples}')
print(f'Total samples that meet the condition 2: {condition_2_total_samples}\n')

# droppin the samples
condition_1_indices_to_drop = condition_1.index
condition_2_indices_to_drop = condition_2.index
df.drop(condition_1_indices_to_drop, inplace=True)
df.drop(condition_2_indices_to_drop, inplace=True)

# check if there is more duplicate values
print("After Drop")
print(f'Dataset size {len(df)}')
print(f'There are {df.duplicated().sum()} duplicated data detected')

Before Drop
Dataset size 17866
There are 1997 duplicated data detected

Data Checking
Total samples that meet the condition 1: 1
Total samples that meet the condition 2: 1

After Drop
Dataset size 15265
There are 0 duplicated data detected


### 2.2 Change NaN Value<a id="2"></a> [![Static Badge](https://img.shields.io/badge/Back_to_Table_of_Contents-orange?style=flat&logo=jupyter&logoColor=white)](#main)

Replacing the following features that have NaN values to 0

- star_rating
- quality_rating
- overall_reviews
- count_reviews
- host_score_review
- count_uploaded_pics
- count_room_types
- count_landmarks
- count_top_attractions
- count_publictransport
- count_closestairports
- count_restaurantscafes

In [4]:
df.star_rating = df.star_rating.fillna(0)
df.quality_rating = df.quality_rating.fillna(0)
df.overall_reviews = df.overall_reviews.fillna(0)
df.count_reviews = df.count_reviews.fillna(0)
df.host_score_review = df.host_score_review.fillna(0)
df.count_uploaded_pics = df.count_uploaded_pics.fillna(0)
df.count_room_types = df.count_room_types.fillna(1)
df.count_landmarks = df.count_landmarks.fillna(0)
df.count_topattractions = df.count_topattractions.fillna(0)
df.count_publictransport = df.count_publictransport.fillna(0)
df.count_closestairports = df.count_closestairports.fillna(0)
df.count_restaurantscafes = df.count_restaurantscafes.fillna(0)
df.rename(columns={'sustainable_level': 'travel_sustainable_level'}, inplace=True)
df['travel_sustainable_level'] = df['travel_sustainable_level'].fillna(0)

### Features Cleaning

#### 2.3.1 About<a id="3"></a> [![Static Badge](https://img.shields.io/badge/Back_to_Table_of_Contents-orange?style=flat&logo=jupyter&logoColor=white)](#main)

- name & description

In [5]:
# keyword on sustainable tourism

def check_eco_mention(description):
  sustainable_terms = [
    'accessible',
    'certification',
    'climate neutral',
    'collaboration',
    'community consent',
    'conservation',
    'continuous improvement',
    'cultural',
    'environmentally sustainable',
    'equitable',
    'fair trade',
    'free roaming wildlife',
    'ghg emissions',
    'greenhouse gas',
    'gstc-certified',
    'gstc-i accredited',
    'gstc-i recognized',
    'habitat',
    'local',
    'low-impact transportation',
    'management',
    'natural heritage',
    'responsible consumption',
    'restoration',
    'sensitivity',
    'social impact assessment',
    'sustainability',
    'sustainable destination strategy',
    'sustainable investment',
    'sustainable materials',
    'sustainable practices',
    'sustainable tourism',
    'threatened species',
    'tourism assets',
    'transparent',
    'water stewardship',
    'wildlife',
    'wildlife interaction',
    'ecofriendly',
    'eco friendly',
    'eco-friendly',
    'eco'
] # source: https://www.gstcouncil.org/gstc-criteria/glossary/
  
  # Check if the value is NaN
  if pd.isna(description):
    return 0

  # Convert the description to lowercase
  description_lower = str(description).lower()

  # Check if any of the specified words are mentioned
  eco_mentioned = any(keyword in description_lower for keyword in sustainable_terms)

  # Return 1 if mentioned, else 0
  return 1 if eco_mentioned else 0

df['eco_mentioned_name'] = df['name'].apply(check_eco_mention)
df['eco_mentioned_description'] = df['description'].apply(check_eco_mention)

- Address

In [6]:
# extract postcode from address feature
def extract_postcode(text):
  if pd.isna(text):
    return None

  text = text.split(',')[-2].strip()
  postcode = re.findall(r'\d+\.*\d*', text)

  if postcode:
    postcode = postcode[0]
    return int(postcode)
  else:
    return None

def add_city_from_postcode(dataframe, csv_file_path):
    # Read the CSV file containing postcode and city columns
    postcode_df = pd.read_csv(csv_file_path)

    # Dictionary mapping postcodes to cities
    postcode_to_city = dict(zip(postcode_df['postcode'], postcode_df['city']))

    # Create a new column 'city' in the original dataframe
    dataframe['city'] = dataframe['extracted_postcode'].apply(lambda x: postcode_to_city.get(x) if pd.notnull(x) else None)

    return dataframe
  
postcode_city = './../references/data/malaysia_all-postcode_multi-sources.csv'

In [7]:
df['extracted_postcode'] = df.address.apply(extract_postcode)
df = add_city_from_postcode(df, postcode_city)

In [8]:
# Filter rows where 'city' column is null
null_postcode_filter = df[df['extracted_postcode'].isnull()]

# Retrieve the addresses of the filtered rows
filtered_data = null_postcode_filter[['state']]

# Print the filtered addresses
print(filtered_data.value_counts())
print(len(filtered_data))

state      
melaka         15
kualalumpur    14
Name: count, dtype: int64
29


In [9]:
# replace the null value in city according to state
df.loc[(df['state'] == 'kualalumpur') & (df['city'].isnull()), 'city'] = 'bukit bintang'
df.loc[(df['state'] == 'melaka') & (df['city'].isnull()), 'city'] = 'banda hilir, melaka raya'

- Coordinate

In [10]:
def extract_coordinates(df):
    # Extract the 'coordinate' column
    coordinates = df['coordinate'].str.split(',', expand=True)

    # Create new columns 'lat' and 'long'
    df['prop_lat'] = coordinates[0].astype(float)
    df['prop_long'] = coordinates[1].astype(float)

    return df

df = extract_coordinates(df)

#### 2.3.2 Room & Price<a id="4"></a> [![Static Badge](https://img.shields.io/badge/Back_to_Table_of_Contents-orange?style=flat&logo=jupyter&logoColor=white)](#main)

In [11]:
def cal_ave_price(row):
    if pd.isna(row):
        return 0  # Return 0 for NaN values
    elif isinstance(row, int):
        return row  # Return the integer as it is
    else:
        # Replace commas with empty strings and split by space to get individual numbers
        numbers = re.sub(r',', '', row).split()
        numbers = [int(num) for num in numbers]  # Convert strings to integers
        numbers.sort()  # Sort the numbers in ascending order
        median = np.median(numbers)  # Calculate median
        return median

df['average_price'] = df['prices_range'].apply(cal_ave_price)

In [12]:
median_by_state_and_star_rating = df.groupby(['state', 'star_rating'])['average_price'].median()

# Perform data imputation for 0 value
df['average_price'] = df.apply(
    lambda row: median_by_state_and_star_rating[(row['state'], row['star_rating'])] if row['average_price'] == 0 else row['average_price'],
    axis=1
)

In [13]:
# remove extreme values
price_outliers = df[(df['average_price'] > 22000)]

# dropping the samples
price_outliers_to_drop = price_outliers.index
df.drop(price_outliers_to_drop, inplace=True)

#### 2.3.3 Review<a id="5"></a> [![Static Badge](https://img.shields.io/badge/Back_to_Table_of_Contents-orange?style=flat&logo=jupyter&logoColor=white)](#main)

- review_titles & reviews_overall_scores

In [14]:
def extract_titles_review_scores(df):
    # Create a dictionary to store extracted values
    extracted_values = {}

    # Iterate over rows
    for index, row in df.iterrows():
        titles = row['review_titles']
        reviews = row['review_overall_scores']

        # Replace NaN with 0 in 'review_titles' and 'review_overall_scores'
        titles = titles if pd.notna(titles) else '0'
        reviews = reviews if pd.notna(reviews) else '0'

        # Split titles and reviews
        title_list = titles.split(', ')
        review_list = list(map(float, reviews.split(', ')))

        # Update the dictionary with extracted values
        for title, review in zip(title_list, review_list):
            # Convert title to lowercase and add "review_" prefix
            title_lower = f"review_{title.lower()}"

            if title_lower not in extracted_values:
                extracted_values[title_lower] = [0] * len(df)

            # Ensure the list has enough elements to accommodate the current index
            while len(extracted_values[title_lower]) <= index:
                extracted_values[title_lower].append(0)

            extracted_values[title_lower][index] = review

    # Add the extracted values to the DataFrame
    for title, values in extracted_values.items():
        # Ensure the length of the values matches the length of the DataFrame
        values = values[:len(df)]
        df[title] = values

    return df

df = extract_titles_review_scores(df)

- score_status

In [15]:
# score_status
# to extract 'Low' or 'High' scores
def extract_scores(text):
    if pd.notna(text):  # check if text is not NaN
        extract_scores = text.split()[0].lower()
        return extract_scores
    else:
        return str('no score')  # return NaN for missing values
    
df['city_score'] = df['score_status'].apply(extract_scores)

- count_qna

In [16]:
# qna
def extract_qna_num(text):
  if pd.notna(text):
    num = text.split()[-1]
    num = num.replace('(',"").replace(')',"")
    return int(num)
  else:
    return 0

df['count_qna'] = df.count_qna.apply(extract_qna_num)

- count_reviewer_by_cat & count_reviewer_by_scores

In [17]:
# count_reviewer_category
def extract_category_counts(df, column_names):
    # Create a dictionary to store extracted values
    extracted_values = {}

    for column_name in column_names:
        # Iterate over rows
        for index, row in df.iterrows():
            category_counts = {}

            if pd.notna(row[column_name]):
                # Use regular expressions to extract category counts
                counts = re.findall(r'\((\d+)\)', row[column_name])
                categories = re.findall(r'([A-Za-z\s]+) \(\d+\)', row[column_name])
                cat_num = re.findall(r'([A-Za-z\s\d+: –]+) \(\d+\)', row[column_name])

                # Determine whether to use 'categories' or 'cat_num'
                cat_list = cat_num if any(char.isdigit() for char in ''.join(cat_num)) else categories

                # Create a dictionary to store the counts for each category
                category_counts = dict(zip(cat_list, map(int, counts)))

                # Fill in missing categories with 0
                for category in set(cat_list):
                    if category not in category_counts and category.lower() != 'all':
                        category_counts[category] = 0

                # Convert values to lowercase and remove white spaces
                category_counts = {key.lower().strip(): value for key, value in category_counts.items() if key.lower().strip() != 'all'}

                # Add "count" to each extracted value
                category_counts = {f'count_{key}': value for key, value in category_counts.items()}

            # Update the dictionary with extracted values
            for title, value in category_counts.items():
                if title not in extracted_values:
                    extracted_values[title] = [0] * len(df)

                # Ensure the list has enough elements to accommodate the current index
                while len(extracted_values[title]) <= index:
                    extracted_values[title].append(0)

                extracted_values[title][index] = value

    # Add the extracted values to the DataFrame
    for title, values in extracted_values.items():
        # Ensure the length of the values matches the length of the DataFrame
        values = values[:len(df)]
        df[title] = values

    return df

cols_to_extract = ['count_reviewer_by_cat', 'count_reviewer_by_scores']
df = extract_category_counts(df, cols_to_extract)

- count_reviewer_by_lan

In [18]:
def process_language_data(df, count_reviewer_by_lan):
    # Read the column
    language_data = df[count_reviewer_by_lan]

    # Initialize new columns
    df['lan_eng_proportion'] = 0.0
    df['lan_malay_proportion'] = 0.0
    df['lan_others_proportion'] = 0.0

    # Lists to store proportions for each row
    eng_proportions = []
    malay_proportions = []
    others_proportions = []

    # Preprocess
    for data in language_data:
        if pd.isna(data):
            eng_proportions.append(0)
            malay_proportions.append(0)
            others_proportions.append(0)
            continue  # Skip if the value is null

        languages = re.findall(r'(\w+) \((\d+)\)', data)
        total_count = int(re.search(r'All \((\d+)\)', data).group(1))

        eng_count, malay_count, others_count = 0, 0, 0

        for language, count in languages:
            proportion = int(count) / total_count * 100

            if language == 'English':
                eng_count = int(count)
            elif language == 'Malay':
                malay_count = int(count)
            else:
                others_count += int(count)

        eng_proportions.append(eng_count / total_count * 100)
        malay_proportions.append(malay_count / total_count * 100)
        others_proportions.append((100 - (eng_count + malay_count) / total_count * 100) if total_count != 0 else 0)

    # Update DataFrame with calculated proportions
    df['lan_eng_proportion'] = eng_proportions
    df['lan_malay_proportion'] = malay_proportions
    df['lan_others_proportion'] = others_proportions

    # Replace NaN values in the specified columns with 0
    df.loc[df[count_reviewer_by_lan].isna(), ['lan_eng_proportion', 'lan_malay_proportion', 'lan_others_proportion']] = 0

    return df

df = process_language_data(df, 'count_reviewer_by_lan')

#### 2.3.4 Host<a id="6"></a> [![Static Badge](https://img.shields.io/badge/Back_to_Table_of_Contents-orange?style=flat&logo=jupyter&logoColor=white)](#main)

- host_count_reviews

In [19]:
# host_count_reviews
def extract_review_and_properties(text):
    if pd.notna(text):
        review = text.split()[2]
        review = review.replace(',', '')
        prop = text.split()[-2]
        prop = prop.replace(',', '')
        return int(review), int(prop)
    else:
        return 0, 0
    
df[['host_count_reviews', 'host_properties']] = df['host_count_reviews'].apply(lambda x: pd.Series(extract_review_and_properties(x)))

#### 2.3.5 Surrounding<a id="7"></a> [![Static Badge](https://img.shields.io/badge/Back_to_Table_of_Contents-orange?style=flat&logo=jupyter&logoColor=white)](#main)

- landmarks, topattractions, publictransport, closestairports & restaurantscafes

In [20]:
# full function
def process_distance_collected(distance_str):
    if pd.isna(distance_str):  # Check if the input is NaN
        return 0

    # Function to convert meters to kilometers
    def convert_to_km(distance_str):
        distances = re.findall(r'(\d+)\s*m', distance_str)
        if not distances:
            return ""  # Return an empty string if no distances are found
        distances_km = [str(float(d) / 1000) for d in distances]  # Convert to kilometers and create a list of strings
        return ", ".join(distances_km)  # Join the distances together with a separator

    # Function to extract distances in kilometers
    def km_extracter(distance_str):
        distances = re.findall(r'(\d+\.\d+|\d+)\s*km', distance_str)
        if not distances:
            return ""  # Return an empty string if no distances are found
        return ", ".join(distances)

    # Function to calculate median distance
    def calculate_median(combined_distances_str):
        if not combined_distances_str:
            return None
        combined_distances = list(map(float, combined_distances_str.split(', ')))
        combined_distances.sort()
        median_index = len(combined_distances) // 2
        return combined_distances[median_index]

    # Apply the functions sequentially
    m_to_km = convert_to_km(distance_str)
    km = km_extracter(distance_str)

    # Combine m_to_km and km into a single string
    combined_distances_str = ", ".join(filter(None, [m_to_km, km]))

    # Calculate median
    median_distance = calculate_median(combined_distances_str)

    return median_distance

df['average_all_landmarks_distances'] = df['all_landmarks_distances'].apply(process_distance_collected)
df['average_all_topattractions_distances'] = df['all_topattractions_distances'].apply(process_distance_collected)
df['average_all_publictransports_distances'] = df['all_publictransports_distances'].apply(process_distance_collected)
df['average_all_closestairports_distances'] = df['all_closestairports_distances'].apply(process_distance_collected)
df['average_all_restaurantscafes_distances'] = df['all_restaurantscafes_distances'].apply(process_distance_collected)

### 2.3.6 Facilities<a id="8"></a> [![Static Badge](https://img.shields.io/badge/Back_to_Table_of_Contents-orange?style=flat&logo=jupyter&logoColor=white)](#main)

In [21]:
def init_new_cats_cols(df, key_values_dict):
    new_columns = {key: [0]*len(df) for key in key_values_dict.keys()}
    for key, values in new_columns.items():
        df[key] = values
    return df

def update_value_cat_cols(df, col_name, key_values_dict):
    for idx, row in df.iterrows():
        if pd.isnull(row[col_name]):
            continue

        steps = set()  # To keep track of unique steps encountered
        for step in [step.lower().replace(' ', '_') for step in row[col_name].split(', ')]:
            if step not in steps:  # Check if step is encountered for the first time
                for key, values in key_values_dict.items():
                    if step in values:
                        df.at[idx, key] += 1
                steps.add(step)  # Add the step to the set of encountered steps
    return df

def update_key_cat_cols(df, col_name, key_values_dict, key_name):
# Initialize count for the specified key if not already present
    if key_name not in df.columns:
        df[key_name] = 0

    # Update count based on key_name
    for idx, row in df.iterrows():
        if pd.notnull(row[col_name]):
            steps = [step.lower().replace(' ', '_') for step in row[col_name].split(', ')]
            if key_name in steps:
                df.at[idx, key_name] += 1

    return df

In [22]:
# source: Booking.com
facilities = {
    'accessibility': [
      'auditory_guidance',
      'emergency_cord_in_bathroom',
      'entire_unit_located_on_ground_floor',
      'entire_unit_wheelchair_accessible',
      'hearing_accessible',
      'upper_floors_accessible_by_elevator',
      'upper_floors_accessible_by_stairs_only'
    ],
    'activities': [
      'aerobics',
      'archery',
      'badminton_equipment',
      'beach',
      'bike_tours',
      'billiards',
      'bingo',
      'bowling',
      'canoeing',
      'cooking_class',
      'cycling',
      'darts',
      'diving',
      'fishing',
      'golf_course_(within_3_km)',
      'happy_hour',
      'hiking',
      'horse_riding',
      "kids'_club",
      'mini_golf',
      'movie_nights',
      'pub_crawls',
      'ski-to-door_access',
      'ski_equipment_hire_on_site',
      'ski_pass_vendor',
      'ski_school',
      'ski_storage',
      'skiing',
      'snorkelling',
      'squash',
      'table_tennis',
      'temporary_art_galleries',
      'tennis_court',
      'tennis_equipment',
      'themed_dinner_nights',
      'tour_or_class_about_local_culture',
      'walking_tours',
      'water_park',
      'water_sport_facilities_on_site',
      'windsurfing'
    ],
    'bathroom': [
      'additional_bathroom',
      'additional_toilet',
      'bath',
      'bath_or_shower',
      'bathrobe',
      'bidet',
      'free_toiletries',
      'hairdryer',
      'private_bathroom',
      'shared_toilet',
      'shower',
      'slippers',
      'toilet',
      'toilet_paper',
      'towels',
      'towels/sheets_(extra_fee)'
    ],
    'bedroom':[
      'alarm_clock',
      'dressing_room',
      'extra_long_beds_(>_2_metres)',
      'linen',
      'wardrobe_or_closet'
    ],
    'building_characteristics': [
      'detached',
      'private_apartment_in_building',
      'semi-detached'
    ],
    'business_facilities': [
      'business_centre',
      'fax',
      'fax/photocopying',
      'meeting/banquet_facilities'
    ],
    'cleaning_services': [
      'daily_housekeeping',
      'dry_cleaning',
      'ironing_service',
      'laundry',
      'trouser_press'
      ],
    'common_areas': [
      'chapel/shrine',
      'games_room',
      'shared_lounge/tv_area'
    ],
    'entertainment_and_family_services': [
      'baby_safety_gates',
      'babysitting/child_services',
      'board_games/puzzles',
      'books',
      'casino',
      'child_safety_socket_covers',
      "children's_playground",
      'dvds',
      'entertainment_staff',
      'evening_entertainment',
      'indoor_play_area',
      'karaoke',
      "kids'_outdoor_play_equipment",
      'live_music/performance',
      'live_sport_events_(broadcast)',
      'nightclub/dj',
      'or_music_for_children',
      'stand-up_comedy'
    ],
    'food_&_drink': [
      'bar',
      'breakfast_in_the_room',
      'coffee_house_on_site',
      'fruits',
      'grocery_deliveries',
      'kid-friendly_buffet',
      'kid_meals',
      'minibar',
      'packed_lunches',
      'restaurant',
      'room_service',
      'snack_bar',
      'special_diet_menus_(on_request)',
      'tea/coffee_maker',
      'vending_machine_(drinks)',
      'vending_machine_(snacks)',
      'wine/champagne'
    ],
    'internet': [
      'internet_available'
    ],
    'kitchen': [
      "children's_high_chair",
      'cleaning_products',
      'coffee_machine',
      'dining_table',
      'dishwasher',
      'electric_kettle',
      'kitchen',
      'kitchenette',
      'kitchenware',
      'microwave',
      'oven',
      'refrigerator',
      'shared_kitchen',
      'stovetop',
      'toaster',
      'tumble_dryer',
      'washing_machine'
    ],
    'languages_spoken': [
      'afrikaans',
      'arabic',
      'bengali',
      'bulgarian',
      'burmese',
      'cantonese',
      'chinese',
      'czech',
      'dutch',
      'english',
      'farsi',
      'filipino',
      'french',
      'georgian',
      'german',
      'gujarati',
      'hausa',
      'hindi',
      'indonesian',
      'irish',
      'italian',
      'japanese',
      'javanese',
      'khmer',
      'korean',
      'lao',
      'macedonian',
      'malay',
      'malayalam',
      'maltese',
      'mandarin',
      'mongolian',
      'norwegian',
      'polish',
      'portuguese',
      'punjabi',
      'romanian',
      'russian',
      'spanish',
      'swedish',
      'tamil',
      'telugu',
      'thai',
      'turkish',
      'ukrainian',
      'urdu',
      'vietnamese'
    ],
    'living_area': [
      'desk',
      'dining_area',
      'fireplace',
      'seating_area',
      'sofa'
    ],
    'media_&_technology': [
      'blu-ray_player',
      'cable_channels',
      'cd_player',
      'computer',
      'dvd_player',
      'flat-screen_tv',
      'game_console',
      'game_console_–_nintendo_wii',
      'game_console_–_ps2',
      'game_console_–_ps3',
      'game_console_–_xbox_360',
      'ipad',
      'ipod_dock',
      'laptop',
      'laptop_safe',
      'pay-per-view_channels',
      'radio',
      'satellite_channels',
      'streaming_service_(like_netflix)',
      'telephone',
      'tv',
      'video',
      'video_games'
    ],
    'miscellaneous': [
      'air_conditioning',
      'allergy-free_room',
      'designated_smoking_area',
      'facilities_for_disabled_guests',
      'family_rooms',
      'heating',
      'higher_level_toilet',
      'lift',
      'lower_bathroom_sink',
      'non-smoking_rooms',
      'non-smoking_throughout',
      'pet_basket',
      'pet_bowls',
      'soundproof_rooms',
      'strollers',
      'toilet_with_grab_rails',
      'visual_aids:_braille',
      'visual_aids:_tactile_signs',
      'wheelchair_accessible'
    ],
    'outdoor_&_view': [
      'city_view',
      'garden_view',
      'inner_courtyard_view',
      'lake_view',
      'landmark_view',
      'mountain_view',
      'pool_view',
      'river_view',
      'sea_view',
      'view'
    ],
    'outdoors': [
      'balcony',
      'barbecue',
      'bbq_facilities',
      'beachfront',
      'garden',
      'outdoor_dining_area',
      'outdoor_fireplace',
      'outdoor_furniture',
      'patio',
      'picnic_area',
      'private_beach_area',
      'private_pool',
      'solarium',
      'sun_terrace',
      'terrace'
    ],
    'parking': [
      'accessible_parking',
      'electric_vehicle_charging_station',
      'parking_garage',
      'street_parking',
      'valet_parking'
    ],
    'pets': [
      'pets_allowed'
    ],
    'reception_services': [
      '24-hour_front_desk',
      'atm/cash_machine_on_site',
      'concierge_service',
      'currency_exchange',
      'express_check-in/check-out',
      'invoice_provided',
      'lockers',
      'luggage_storage',
      'private_check-in/check-out',
      'tour_desk'
    ],
    'room_amenities': [
      'carpeted',
      'clothes_rack',
      'drying_rack_for_clothing',
      'electric_blankets',
      'fan',
      'fold-up_bed',
      'hardwood_or_parquet_floors',
      'hot_tub',
      'hypoallergenic',
      'interconnected_room(s)_available',
      'iron',
      'ironing_facilities',
      'mosquito_net',
      'pants_press',
      'private_entrance',
      'socket_near_the_bed',
      'sofa_bed',
      'soundproofing',
      'tile/marble_floor'
    ],
    'safety_&_security': [
      '24-hour_security',
      'carbon_monoxide_detector',
      'cctv_in_common_areas',
      'cctv_outside_property',
      'fire_extinguishers',
      'key_access',
      'key_card_access',
      'safety_deposit_box',
      'security_alarm',
      'smoke_alarms'
    ],
    'services_&_extras': [
      'executive_lounge_access',
      'wake-up_service',
      'wake_up_service/alarm_clock'
    ],
    'shops': [
      'barber/beauty_shop',
      'minimarket_on_site'
    ],
    'swimming_pool': [
      'adult_only',
      'all_ages_welcome',
      'fence_around_pool',
      'heated_pool',
      'hot_spring_bath',
      'infinity_pool',
      'open_all_year',
      'opening_times',
      'plunge_pool',
      'pool/beach_towels',
      'pool_bar',
      'pool_cover',
      'pool_is_on_rooftop',
      'pool_with_view',
      'salt-water_pool',
      'seasonal',
      'shallow_end',
      'suitable_for_kids',
      'sun_loungers_or_beach_chairs',
      'sun_umbrellas',
      'swimming_pool_toys',
      'water_slide'
    ],
    'transport': [
      'airport_shuttle',
      'bicycle_rental',
      'car_hire',
      'public_transport_tickets',
      'shuttle_service'
    ],
    'wellness': [
      'back_massage',
      'beauty_services',
      'body_scrub',
      'body_treatments',
      'body_wrap',
      'couples_massage',
      'facial_treatments',
      'fitness',
      'fitness/spa_locker_rooms',
      'fitness_centre',
      'fitness_classes',
      'foot_bath',
      'foot_massage',
      'full_body_massage',
      'hair_colouring',
      'hair_cut',
      'hair_styling',
      'hair_treatments',
      'hammam',
      'hand_massage',
      'head_massage',
      'hot_tub/jacuzzi',
      'kids\'_pool',
      'light_therapy',
      'make_up_services',
      'manicure',
      'massage',
      'massage_chair',
      'neck_massage',
      'open-air_bath',
      'pedicure',
      'personal_trainer',
      'public_bath',
      'sauna',
      'spa/wellness_packages',
      'spa_and_wellness_centre',
      'spa_bath',
      'spa_facilities',
      'spa_lounge/relaxation_area',
      'steam_room',
      'sun_loungers_or_beach_chairs',
      'waxing_services',
      'yoga_classes'
    ]
}

In [23]:
df = init_new_cats_cols(df, facilities)
df = update_value_cat_cols(df, 'sub_facilities', facilities)
df = update_key_cat_cols(df, 'facilities', facilities, 'pets')
df = update_key_cat_cols(df, 'facilities', facilities, 'parking')
df = update_key_cat_cols(df, 'facilities', facilities, 'internet')

#### 2.3.7 Sustainability<a id="9"></a> [![Static Badge](https://img.shields.io/badge/Back_to_Table_of_Contents-orange?style=flat&logo=jupyter&logoColor=white)](#main)

- sustainable_level

In [24]:
def create_sustainable_label(level):
    if level == 0:
        return 'non-travel sustainable'
    else:
        return 'travel sustainable'
    
df['sustainable_label'] = df['travel_sustainable_level'].apply(lambda x: create_sustainable_label(x))

- sustainable impact category

In [25]:
# source: Booking.com
impact_categories = {
    'energy_and_greenhouse_gases': [
        "most_lighting_throughout_property_uses_energy-efficient_led_bulbs",
        "all_windows_are_double-glazed",
        "vegetarian_menu_options_offered",
        "vegan_menu_options_offered",
        "key_card_or_motion-controlled_electricity",
        "100%_renewable_electricity_used_throughout",
        "the_property_makes_efforts_to_reduce_their_food_wastage",
        "most_food_provided_at_the_property_is_locally_sourced",
        "electric_car_charging_station"
    ],
    'waste': [
        "single-use_plastic_miniature_shampoo_conditioner_and_body_wash_bottles_not_used",
        "single-use_plastic_water_bottles_not_used",
        "water_cooler/dispenser",
        "single-use_plastic_beverage_bottles_not_used",
        "single-use_plastic_cutlery/plates_not_used",
        "single-use_plastic_cups_not_used",
        "single-use_plastic_straws_not_used",
        "single-use_plastic_stirrers_not_used",
        "recycling_bins_available_to_guests_and_waste_is_recycled"
    ],
    'destination_and_community': [
        "local_artists_are_offered_a_platform_to_display_their_talents",
        "tours_and_activities_organised_by_local_guides_and_businesses_offered",
        "invests_a_percentage_of_revenue_back_into_community_projects_or_sustainability_projects",
        "provides_guests_with_information_regarding_local_ecosystems,_heritage_and_culture,_as_well_as_visitor_etiquette"
    ],
    'water': [
        "water-efficient_toilets",
        "water-efficient_showers",
        "option_to_reuse_towels",
        "option_to_opt-out_of_daily_room_cleaning"
    ],
    'nature': [
        "wild_(non-domesticated)_animals_are_not_displayed/interacted_with_while_captive_on_the_property_or_harvested,_consumed,_or_sold.",
        "offsets_a_portion_of_their_carbon_footprint",
        "most_food_provided_is_organic",
        "green_spaces_such_as_gardens/rooftop_gardens_on_the_property"
    ]
}

In [26]:
df = init_new_cats_cols(df, impact_categories)
df = update_value_cat_cols(df, 'sustainable_steps_taken', impact_categories)

### 2.4 Finalising Dataset<a id="10"></a> [![Static Badge](https://img.shields.io/badge/Back_to_Table_of_Contents-orange?style=flat&logo=jupyter&logoColor=white)](#main)

2.4.1 Dropping the original features

In [27]:
# dropping the original features
cols_to_drop = [
  'name',
  'address',
  'coordinate',
  'description',
  'prices_range',
  'review_titles',
  'review_overall_scores',
  'score_status',
  'count_reviewer_by_cat',
  'count_reviewer_by_scores',
  'count_reviewer_by_lan',
  'all_landmarks_distances',
  'all_topattractions_distances',
  'all_publictransports_distances',
  'all_closestairports_distances',
  'all_restaurantscafes_distances',
  'count_facilities',
  'count_sub_facilities',
  'facilities',
  'sub_facilities',
  'count_sustainable_steps_title',
  'sustainable_steps_title',
  'count_sustainable_steps_taken',
  'sustainable_steps_taken',
  'extracted_postcode',
  'review_0'
  ]

df = df.drop(cols_to_drop, axis=1)
print(df.shape)

(15263, 84)


2.4.2 Move the target feature to the end

In [28]:
# move the target to the end
# Column to move to the last position
column_to_move = 'sustainable_label'

# Reorder the columns
new_columns = [col for col in df.columns if col != column_to_move] + [column_to_move]
df = df[new_columns]

2.4.3 Final check for null values

In [29]:
# check every single feature if there any existence of null or NaN values
null_columns = df.columns[df.isnull().any(axis=0)]
if len(null_columns) > 0:
    print("DataFrame contains null or NaN values in the following columns:")
    for column in null_columns:
        print(column)
else:
    print("DataFrame does not contain null or NaN values")

DataFrame does not contain null or NaN values


2.4.4 Saving the processed data

In [30]:
save_dir = './../data/processed/processed_data.csv'
df.to_csv(save_dir, index=False)

### 2.5 Process Dataset for Modeling<a id="11"></a> [![Static Badge](https://img.shields.io/badge/Back_to_Table_of_Contents-orange?style=flat&logo=jupyter&logoColor=white)](#main)

2.5.1 Encoding categorical variables

In [31]:
def process_dataset(df):
    # Define the mapping inside the function
    mapping = {
        'travel_sustainable_level': {
            0: 0,
            'Travel Sustainable Level 1': 1,
            'Travel Sustainable Level 2': 2,
            'Travel Sustainable Level 3': 3,
            'Travel Sustainable Level 3+': 4
        },
        'city_score': {
            'no score': 0,
            'low': 1,
            'high': 2
        },
        'sustainable_label': {
            'non-travel sustainable': 0,
            'travel sustainable': 1
        }
    }

    # Normalize and encode 'city' column
    city_frequencies = df['city'].value_counts(normalize=True)
    city_encoding = df['city'].map(city_frequencies)
    df['city'] = city_encoding

    # Normalize and encode 'state' column
    state_frequencies = df['state'].value_counts(normalize=True)
    state_encoding = df['state'].map(state_frequencies)
    df['state'] = state_encoding

    # Apply mappings to the specified features
    for feature, map_dict in mapping.items():
        df[feature] = df[feature].map(lambda x: map_dict.get(x, x))  # Preserve original value if not found in mapping

    # Ensure 'travel_sustainable_level' is of type int
    df['travel_sustainable_level'] = df['travel_sustainable_level'].astype(int)

    # print process completed
    print('Dataset successfully processed!\n')

    return df

processed_df = process_dataset(df)

Dataset successfully processed!



2.5.2 Handling outliers

In [33]:
def winsorize_columns(df, columns, limits=(0.05, 0.05)):
    """
    Apply Winsorizing to specific columns in a DataFrame.

    Parameters:
        df (pandas.DataFrame): DataFrame containing the columns to be winsorized.
        columns (list of str): Names of the columns to be winsorized.
        limits (tuple of float): Lower and upper limits of the winsorization.
                                  Values outside the limits will be replaced.

    Returns:
        pandas.DataFrame: DataFrame with winsorized values for specified columns.
    """
    winsorized_df = df.copy()
    for column in columns:
        winsorized_df[column] = mstats.winsorize(df[column], limits=limits)
    return winsorized_df

In [34]:
cols_with_outliers = [
    "count_room_types",
    "count_reviews",
    "host_score_review",
    "host_count_reviews",
    "count_landmarks",
    "count_closestairports",
    "count_restaurantscafes",
    "average_price",
    "count_families",
    "count_couples",
    "count_groups of friends",
    "count_solo travellers",
    "count_business travellers",
    "count_superb: 9+",
    "count_good: 7 – 9",
    "count_passable: 5 – 7",
    "count_poor: 3 – 5",
    "count_very poor: 1 – 3",
    "lan_malay_proportion",
    "lan_others_proportion",
    "host_properties",
    "average_all_landmarks_distances",
    "average_all_topattractions_distances",
    "average_all_closestairports_distances",
    "average_all_restaurantscafes_distances",
    "accessibility",
    "activities",
    "bedroom",
    "building_characteristics",
    "business_facilities",
    "cleaning_services",
    "common_areas",
    "entertainment_and_family_services",
    "food_&_drink",
    "kitchen",
    "languages_spoken",
    "media_&_technology",
    "miscellaneous",
    "outdoor_&_view",
    "outdoors",
    "parking",
    "reception_services",
    "room_amenities",
    "services_&_extras",
    "swimming_pool",
    "transport",
    "wellness",
    "energy_and_greenhouse_gases",
    "destination_and_community",
    "nature"
]

In [35]:
winsorized_df = winsorize_columns(processed_df, cols_with_outliers)

2.5.3 Saving the processed data

In [36]:
save_model_dir = './../data/modeling/modeling_data.csv'
winsorized_df.to_csv(save_model_dir, index=False)